# Basics of WSI processing

 This notebook will guide you through the basics of WSI processing.
 Because of the size of the data/models this course would like to introduce, note that
 most of the heavy computations have been done in advance: training, embeddings; LLM are run in the cloud.
 This TP will focus on using the models, understanding their capabilities and limitations, rather than training them.

 At the end of this notebook, you should be familiar with:
 - Handling WSI data and understanding their pyramidal structure.
 - Extract tiles from a WSI, at any resolution, only in the tissue regions.
 - Get an intuition about tiles embeddings.


**_IMPORTANT_** First thing: If you have not done it yet, download this https://drive.google.com/file/d/1VN57GS0d-fVQkBlc63UC7jVXxnw4Dw_W/view?usp=sharing and untar it in this folder.
You should now have an `./assets` subfolder.

**_IMPORTANT BIS_** Second, please download the following tar and untar it in your `./assets` folder! 
These are useful data that I forgot to put in the initial tar

LINK : https://drive.google.com/file/d/1pX_Ai2rXVnPhk4vdk8cEqoQWeY57osn7/view?usp=sharing 

> 👀​ How to use these notebooks ?
> Go through the notebooks cells by cells, they are not independant and you will need previous cells to run the later ones.
> Many questions are scattered throughout the notebooks. Questions in the first notebook are interesting to get used to WSI manipulation, 
> questions in the second may be a bit broader and open: you are free to solve these questions yourself (what I would advise) or to just copy the answers present in the `answers.py` file.
> Open questions will be adressed orally. Alternatively, you may find the full corrected notebook in the first commit of this repo.

In [ ]:
import os
import math
import subprocess
from pathlib import Path
import itertools
import json
import random

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import open_clip
import openslide
import torch
import einops
from PIL import Image
from rich import print as rprint
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.morphology import closing, opening, footprint_rectangle
from tqdm import tqdm
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

In [6]:
wsi_path = "./assets/TCGA-Z7-A8R6-01Z-00-DX1.CE4ED818-D762-4324-9DEA-2ACB38B9B0B9.svs"
wsi_name = Path(wsi_path).stem
wsi = openslide.open_slide(wsi_path)
wsi.get_thumbnail((512, 512)).show()


## Understanding the Pyramidal Structure of WSIs

Whole Slide Images are stored in a specialized pyramidal format that optimizes their accessibility and viewing. This structure consists of multiple resolution levels, where each level represents a progressively downsampled version of the original image. Each level is stored on disk: it increases the global weight of the object but make it easier to manipulate it.

 Level 0 is the original resolution - very often corresponding to a 40x magnification (around 0.25 microns per pixel).
 The downsampling factor between each level is usually 2, but may vary depending on the scanner used - 
 many of the slides of the TCGA, scanned on an Aperio scanner, have a downsampling factor of 4.

 Because maximum resolution and downsampling factors may depend on the scanner, there is not a one-one correspondance between a slide *level* and a *magnification* - finding this correspondance is left to the user (or software) given the metadata.

 You can access all these information using the OpenSlide object.

In [ ]:
console = Console()

# Create a table for WSI information
table = Table(title="WSI Information")
table.add_column("Property", style="cyan")
table.add_column("Value", style="green")
table.add_row("Level count", str(wsi.level_count))
table.add_row("Level dimensions", str(wsi.level_dimensions))
table.add_row("Level downsamples", str(wsi.level_downsamples))
console.print(table)


In [ ]:

dimensions = wsi.dimensions
console.print(Panel(f"Slide dimensions: {dimensions}", title="Slide Size"))

properties_table = Table(title="Slide Properties")
properties_table.add_column("Property", style="cyan")
properties_table.add_column("Value", style="green")

for key, value in wsi.properties.items():
    properties_table.add_row(key, str(value))

console.print(properties_table)


In [ ]:
# Let's visualize this a bit !
# let's try to pick a region at the center of the slide (to have sone tissue in it).

center_x = dimensions[0] // 2
center_y = dimensions[1] // 2
region_size = (256, 256)

region_high = wsi.read_region(
    (center_x - region_size[0]//2, center_y - region_size[1]//2),
    0,
    region_size
)

region_mid = wsi.read_region(
    (center_x, center_y),
    1,
    region_size
)

region_low = wsi.read_region(
    (center_x, center_y),
    2,
    region_size
)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(region_high)
axes[0].set_title('Level 0 (High Resolution)')
axes[0].axis('off')
axes[1].imshow(region_mid)
axes[1].set_title('Level 1 (Medium Resolution)')
axes[1].axis('off')
axes[2].imshow(region_low)
axes[2].set_title('Level 2 (Low Resolution)')
axes[2].axis('off')
plt.tight_layout()
plt.show()

> **Question 1.1**: Resolution Analysis
> Using the OpenSlide object properties, determine:
> 1. The resolution of each pyramid level
> 2. Express the results in terms of:
>    - Magnification (e.g., 40x, 20x, etc.)
>    - Microns per pixel (μm/pixel)

In [ ]:
# Answer
def get_magnification(level):
   pass 

> **Question 1.2**: Image Scale Analysis
> Calculate the size of the highest resolution level (Level 0):
> - Determine the total number of pixels
> - Consider why working with the full resolution directly might be challenging
> - Think about memory implications when processing such large images

In [ ]:
# Answer

## STEP 1: Tiling Strategy for WSI Analysis

### Why Tiling?
As demonstrated by our previous calculations, working with the entire WSI at full resolution is computationally impractical due to its massive size. Instead, we'll break down the image into manageable tiles for processing.

### Approach
- **Grid Generation**: Create a systematic grid of tiles across the WSI
- **Tissue Detection**: Filter tiles to keep only those containing relevant tissue
- **Encoding**: encode the tile information using pre-trained models

### Implementation Note
We'll use OpenSlide for this implementation, though modern alternatives like CuCIM exist. 

In [15]:
# Configuration for tiling
tile_size = 256  # Final tile size for deep learning models
magnification_tile = 10  # Target magnification (10x)
mask_tolerance = 0.9  # Tissue detection threshold (90% tissue required)

# Calculate which pyramid level to use for the target magnification
mag_level0 = 40  # get that from the slide object. 
ds_per_level = {'.svs': 4, '.ndpi': 2, '.tiff': 2, '.tif': 2}  # Downsampling factors - it can indeed depend on the scanner. To simplify, because tcga are svs and most slides are taken using a 4x downsampling Aperio scanner, I shortcut svs: x4.
ext = Path(wsi_path).suffix


We want to sample a tile at a given magnification (for insance, we want to extract tiles at 10x) 
> **_Question 1.3_** Given the magnification at level 0 and the downsampling factor, compute the level for extracting the slide.

In [ ]:
#Answer

### Tissue Detection Strategy

Our goal is to extract tiles that contain meaningful tissue content while excluding background areas. We'll implement this using a two-step approach:

1. **Tissue Mask Creation**
   - Use Otsu thresholding to separate tissue from background
   - Apply morphological operations to clean up the mask

2. **Tile Filtering**
   - Calculate tissue content percentage for each tile
   - Keep tiles exceeding a specified tissue threshold

**Implementation Note**: This is a basic, handcrafted approach to tissue detection. In practice, you might want to consider:
 - More sophisticated tissue detection algorithms
 - Deep learning-based tissue classifiers
 - Additional filtering for artifacts and scanning errors
 - Domain-specific adaptations for different tissue types

 An simple alternative to this masking strategy is to design a tile classifier to exclude background tiles.
 It can be as simple as a thresold on the spatial average of the pixel values, for instance.

In [ ]:
def make_auto_mask(thumbnail):
    """Create a binary mask from thumbnail using Otsu algorithm and morphological operations."""
    im = np.array(thumbnail)[:, :, :3]
    im_gray = rgb2gray(im)
    size = im_gray.shape
    im_gray = im_gray.flatten()
    
    pixels_int = im_gray[np.logical_and(im_gray > 0.02, im_gray < 0.98)]
    
    if len(pixels_int) == 0:
        return np.zeros(size, dtype=bool)
    
    # Apply Otsu thresholding
    threshold = threshold_otsu(pixels_int)
    mask = (im_gray < threshold).reshape(size)
    
    # Apply morphological operations to clean up the mask
    mask = opening(closing(mask, footprint_rectangle((2,2))), footprint_rectangle((2,2)))
    return mask

# Get thumbnail and create tissue mask
thumbnail = wsi.get_thumbnail(wsi.level_dimensions[-1])  # Use lowest resolution level
tissue_mask = make_auto_mask(thumbnail)

# Visualize tissue detection
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(thumbnail)
axes[0].set_title('Original Thumbnail')
axes[0].axis('off')

axes[1].imshow(tissue_mask)
axes[1].set_title(f'Tissue Mask ({tissue_mask.sum()/tissue_mask.size*100:.1f}% tissue)')
axes[1].axis('off')

plt.tight_layout()
plt.show()


The idea now is to create a grid (i.e. a set of coordinates that regularly span the image), and check, for each tile of the grid, how much falls into the mask.

> **_Question 1.4_** Create this grid of tiles! They can be overlapping or not - You could add this as a parameter.

In [18]:
# Answer
def grid_coords(slide, point_start, point_end, patch_size):
    pass

There is now several ways to check if the tile falls into the mask.
Either downsample the tile to the mask level or the reverse. 
> **_Question 1.5_** Implement one of these solutions.

In [19]:
# Answer
def check_coordinates(row, col, patch_size, mask, mask_downsample, mask_tolerance=0.9):
    pass

In [ ]:
# Generate the complete grid -  we generate it at resolution 0
image_height, image_width = wsi.dimensions[1], wsi.dimensions[0]
mask_downsample = int(wsi.level_downsamples[-1])  # Downsample factor for mask level
size_at_0 = tile_size * (ds_per_level[ext] ** level_tile)  # Tile size at level 0 coordinates

# Create uniform grid
all_coordinates = grid_coords(wsi, (0, 0), (image_height, image_width), (size_at_0, size_at_0))

console.print(f"Generated {len(all_coordinates):,} potential tile locations")


In [ ]:
# Step 3: Filter grid based on tissue content
valid_tiles = []
for row, col in all_coordinates:
    if check_coordinates(row, col, (size_at_0, size_at_0), tissue_mask, mask_downsample, mask_tolerance):
        valid_tiles.append((row, col, size_at_0, size_at_0))

console.print(Panel(f"Grid Generation Results:\n• Total potential tiles: {len(all_coordinates):,}\n• Valid tiles (≥{mask_tolerance*100:.0f}% tissue): {len(valid_tiles):,}\n• Tissue proportion: {len(valid_tiles)/len(all_coordinates)*100:.1f}%", title="Tiling Results"))


Let's visualize where the tiles are localized.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 6))

# Show tissue mask with valid tile locations
ax.imshow(thumbnail)
ax.set_title('WSI Thumbnail + Tiling Grid (sparse)')

# we will show only a subset of the tiles - for visibiity
sparse_tiles = valid_tiles[::max(1, len(valid_tiles)//200)]
for row, col, height, width in sparse_tiles:
    mask_row = row // mask_downsample
    mask_col = col // mask_downsample
    mask_height = height // mask_downsample
    mask_width = width // mask_downsample
    
    rect = patches.Rectangle((mask_col, mask_row), mask_width, mask_height,
                           linewidth=1.5, edgecolor='lime', facecolor='none', alpha=0.9)
    ax.add_patch(rect)

ax.axis('off')
plt.tight_layout()
plt.show()


... And directly some of the tiles themselves.

In [ ]:
def extract_tile(wsi, row, col, level, tile_size):
    """Extract a single tile from the WSI."""
    return wsi.read_region(
        location=(col, row), 
        level=level, 
        size=(tile_size, tile_size)
    ).convert('RGB')

n_sample_tiles = min(9, len(valid_tiles))
sample_indices = np.random.choice(len(valid_tiles), n_sample_tiles, replace=False)
sample_tiles = [valid_tiles[i] for i in sample_indices]

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
fig.suptitle(f'Sample Extracted Tiles (Level {level_tile}, {magnification_tile}x magnification)', fontsize=14)

for i, (row, col, height, width) in enumerate(sample_tiles):
    ax = axes[i//3, i%3]
    tile = extract_tile(wsi, row, col, level_tile, tile_size)
    
    ax.imshow(tile)
    ax.set_title(f'Tile {i+1}\n({row//1000}k, {col//1000}k)', fontsize=10)
    ax.axis('off')

for i in range(n_sample_tiles, 9):
    axes[i//3, i%3].axis('off')
plt.tight_layout()
plt.show()
console.print(Panel(f"Successfully generated tiling grid!\n• Ready for tile extraction and processing\n• Grid contains {len(valid_tiles):,} valid tile locations\n• Each tile: {tile_size}×{tile_size} pixels at {magnification_tile}× magnification", title="Tiling Complete"))


## Tile Embeddings with Pathology Foundation Models

The second step is to extract features from pathology tiles, to then be aggregated at the slide level.

### Evolution of WSI Feature Extraction
The field of digital pathology has seen significant advancement in how we process and analyze WSI tiles.
Before 2020 we used to use ImageNet-pretrained encoders.
This strategy worked (surprisingly) well but was limited by the gap between natural images and histology.

2020 has seen the rise of SSL (summer 2020 is the summer of SimCLR, MoCo, SimSiam etc...), which found in pathology a perfect application:
we have a quasi unlimited amount of tiles but no labels!
This strategy leverages self-supervised learning, trained on millions of pathology images.

### Available Foundation Models
Several state-of-the-art models are readily available on HuggingFace:

| Model | Description | Link |
|-------|-------------|------|
| H-Optimus-0/1 | models trained on one of the biggest WSI dataset | [HuggingFace](https://huggingface.co/bioptimus/) |
| UNI2 | on par with bioptimus models | [HuggingFace](https://huggingface.co/MahmoodLab/UNI2-h) |
| Gigapath | They implement a rather basic tile-encoder (that works well) but mostly a slide encoder | [HuggingFace](https://huggingface.co/prov-gigapath/prov-gigapath) |

### Current Landscape
- New models are released frequently
- Development requires significant computational resources i.e. they are led by major research labs and corporations
- **Best Practice**: Use these pre-trained models out-of-the-box
  - Proven strong performance on various downstream tasks
  - Efficient transfer learning capabilities


In [ ]:
# Embedding here is done using a pathology CLIP model, which will be described in greater details in the second notebook.
# This is a particular type of image foundation model, trained with the supervision of text.
def embed_batch(wsi, coordinates, model, preprocess):
    tiles = [extract_tile(wsi, row, col, level_tile, tile_size) for row, col, _, _ in coordinates]
    tiles = [preprocess(tile) for tile in tiles]
    tiles = torch.stack(tiles)
    tiles = model.encode_image(tiles)
    return tiles

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='assets/pathgenclip.pt')
batch_size = 20
valid_tiles_batched = [valid_tiles[i:i+batch_size] for i in range(0, len(valid_tiles), batch_size)]

embeddings = []
for tile in tqdm(valid_tiles_batched):
    embeddings.append(embed_batch(wsi, tile, model, preprocess))

 If you can't run that on your machine (this might take some time!), you could try:
 - enabling GPU acceleration if you got any (it would work on mac mps accelerator)
 - embed a **random** subset of the tiles
 - just use the pre-computed embeddings available here:

In [25]:
# Embeddings is here a matrix of size (n_tiles, 768)
# Coordinates is a list of tuples (row, col, height, width) of size (n_tiles, 4)
embeddings = np.load("./assets/embeddings_tiles.npy")
with open('./assets/coordinates_tiles.json', 'r') as f:
    coordinates = json.load(f)

> Question 1.6: Analyzing Embedding Space Structure

> The latent space of tile embeddings (the high-dimensional manifold where embeddings reside) should be organized according to semantic similarity between images - if the pre-training was successful.
> 
> **Task**: Let's verify this hypothesis through clustering analysis:
> 1. Use K-means to cluster the embeddings
> 2. Visualize the clusters using t-SNE dimensionality reduction
> 3. Extract and examine example tiles from each cluster
> 4. Analyze whether tiles within clusters share meaningful visual/semantic properties

In [ ]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Answer

The next challenge in computational pathology is to effectively aggregate these tile-level features to make a slide-level prediction. As pathologists know, critical diagnostic information often arises from the broader context, such as tumoral heterogeneity, the morphology of the invasive front, and the spatial arrangement of different tissue types.

Most modern approaches to WSI analysis are inspired by Multiple Instance Learning (MIL). This framework treats each WSI as a "bag" of tile instances, where the overall slide label is determined by the properties of the tiles within it. A key advantage of MIL is its permutation invariance—the order of tiles in the bag doesn't matter.

The seminal paper in this area, Attention-based MIL (abMIL), remains a powerful and relevant baseline today. The core idea is that not all tiles in the bag are equally important for the final diagnosis. For instance, tiles containing healthy fibrotic tissue might be irrelevant to a tumor classification task.

AbMIL introduces an attention mechanism that learns to assign an importance score to each tile. A small neural network (often a simple two-layer MLP) computes these attention scores from the tile embeddings. The final slide representation is then created by taking a weighted average of all tile embeddings, guided by their learned attention. This allows the model to focus on the most informative regions of the slide.

#### Further Reading & Advanced Models
The field has evolved rapidly, with many powerful architectures building upon this foundation. Here are a few key papers for further reading:

##### Seminal MIL Papers:
**abMIL**: Ilse, M., Tomczak, J., & Welling, M. (2018). [ Attention-based Deep Multiple Instance Learning ](https://arxiv.org/abs/1802.04712).

**dsMIL**: Li, B., Li, Y., & Eliceiri, K. W. (2021). [ Dual-stream Multiple Instance Learning Network for Whole Slide Image Classification ](https://arxiv.org/abs/2011.08939).

**CLAM**: Lu, M. Y., Williamson, D. F., Chen, T. Y., Chen, R. J., Barbieri, M., & Mahmood, F. (2021). [ Data-Efficient and Weakly Supervised Computational Pathology on Whole-Slide Images ](https://www.nature.com/articles/s41551-020-00682-w).

##### Slide-Level Foundation Models:
More recently, work trained slide-level representations with self-supervision, giving rise to slide-foundation level, that can be used as-is for many downstream tasks.

**Giga-SSL**: Lazard, T., Lerousseau, M., Decencière, E., & Walter, T. (2023). [ Giga-SSL: Self-Supervised Learning for Gigapixel Images ](https://arxiv.org/abs/2212.03273).

**GigaPath**: Rao, A., et al. (2024). [ GigaPath: A Million-Slide Foundation Model for Computational Pathology ](https://www.nature.com/articles/s41586-024-07441-w).

**PANTHER**: Singhal, K., et al. (2024). [ Pan-Cancer Histology-Genomic Integration at Scale ](https://arxiv.org/abs/2405.11643).
